In [59]:
import numpy as np
import pandas as pd
import tqdm
from sklearn import tree
from kmodes.kmodes import KModes
from sklearn.cluster import SpectralClustering
from collections import Counter
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.cluster import AffinityPropagation
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
import math 
import sklearn
import scipy.special as sp
from sklearn.cluster import Birch
from tqdm import tqdm
import SimpSOM as sps
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import FastICA
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from itertools import combinations
%matplotlib notebook

df=[]
df1=[]
#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/b"+str(i)+".csv"))

#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/f2"+str(i)+".csv"))

for i in range(7):
    df.append(pd.read_csv("/home/franc/franc/matches/ec"+str(i)+".csv"))

#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/n"+str(i)+".csv"))

#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/scp"+str(i)+".csv"))

#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/d2"+str(i)+".csv"))
    
#for i in range(6):
#    df.append(pd.read_csv("/home/franc/franc/matches/d1"+str(i)+".csv"))    

#df1.append(pd.read_csv("/home/franc/franc/matches/SWE.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/DNK.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/CHN.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/ARG.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/NOR.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/JPN.csv"))#Doesnt work well
#df1.append(pd.read_csv("/home/franc/franc/matches/AUT.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/POL.csv"))

df=pd.concat(df,sort=False)
#df1=pd.concat(df1,sort=False)
rat_dic={}
fet_set=[]
tar_set=[]

In [64]:
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * ((rating1) - rating2) / 400)) 
  
  
# Function to calculate Elo rating 
# K is a constant. 
# d determines whether 
# Player A wins or Player B.  
def EloRating(Ra, Rb, K, d): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = nprob(Ra, Rb) 
    Pb = 1-Pa
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if (d == 1) : 
        Ra = Ra + K * (1 - Pa) 
        Rb = Rb + K * (0 - Pb) 
      
  
    # Case -2 When Player B wins 
    # Updating the Elo Ratings 
    elif (d==0.5) : 
        Ra = Ra + K * (0.5 - Pa) 
        Rb = Rb + K * (0.5 - Pb) 
      
    else : 
        Ra = Ra + K * (0 - Pa) 
        Rb = Rb + K * (1 - Pb) 
      
    return Ra,Rb
    print("Updated Ratings:-") 
    print("Ra =", round(Ra, 6)," Rb =", round(Rb, 6))

def Probabilityad(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * ((rating1) - rating2) / 400)) 

def nprob(rating1, rating2): 
  
    return 1/2*(1 + sp.erf((rating1-rating2)/(300*math.sqrt(2)))) 

def spreadelo(Ra, Rb, K, ga,gb,n): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
    Pa = nprob(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pb = 1-Pa 
  
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if ga-gb > n:
        Ra=Ra+K*(1-Pa)
        Rb=Rb+K*(0-Pb)
    else:
        Ra=Ra+K*(0-Pa)
        Rb=Rb+K*(1-Pb)
      
    return Ra,Rb

def adelo(Ra, Rb, K, g,n): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
    Pa = nprob(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pb = 1-Pa 
  
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if g > n:
        Ra=Ra+K*(1-Pa)
        Rb=Rb+K*(0-Pb)
    else:
        Ra=Ra+K*(0-Pa)
        Rb=Rb+K*(1-Pb)
      
    return Ra,Rb

def kgvelo(Ra, Rb, K, ga,gb): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
    Pa = nprob(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pb = 1-Pa 
  
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if ga==0:
        Ra=Ra+K*(0-Pa)
    else:
        Ra=Ra+K*(1-Pa)
        
    if gb==0:
        Rb=Rb+K*(0-Pb)
    else:
        Rb=Rb+K*(1-Pb)
      
    return Ra,Rb

In [65]:
df['Date']=pd.to_datetime(df['Date']
                              ,infer_datetime_format=True)
last_two=df[(df["Date"].dt.year>2012) & (df["Date"].dt.year<2018)]
for i in df.HomeTeam.unique():
    rat_dic[i]={"tr":1500,"tr1":1500,"tr-1":1500,"tr2":1500,"tr-2":1500,"ar":1500,"ar1":1500,"ar2":1500,"dr":1500,"dr1":1500,"dr2":1500,"kgv":1500,"nm":0}

test=df[(df["Date"].dt.year>=2014) & (df["Date"].dt.year<2019)]

for i in range(len(last_two)):
    rat_dic[last_two.iloc[i,2]]["tr"],rat_dic[last_two.iloc[i,3]]["tr"]=spreadelo(rat_dic[last_two.iloc[i,2]]["tr"],rat_dic[last_two.iloc[i,3]]["tr"],60,last_two.iloc[i,4],last_two.iloc[i,5],0)
    rat_dic[last_two.iloc[i,2]]["tr1"],rat_dic[last_two.iloc[i,3]]["tr-1"]=spreadelo(rat_dic[last_two.iloc[i,2]]["tr1"],rat_dic[last_two.iloc[i,3]]["tr-1"],60,last_two.iloc[i,4],last_two.iloc[i,5],1)
    rat_dic[last_two.iloc[i,2]]["tr2"],rat_dic[last_two.iloc[i,3]]["tr-2"]=spreadelo(rat_dic[last_two.iloc[i,2]]["tr2"],rat_dic[last_two.iloc[i,3]]["tr-2"],60,last_two.iloc[i,4],last_two.iloc[i,5],2)
    rat_dic[last_two.iloc[i,3]]["tr1"],rat_dic[last_two.iloc[i,2]]["tr-1"]=spreadelo(rat_dic[last_two.iloc[i,3]]["tr1"],rat_dic[last_two.iloc[i,2]]["tr-1"],60,last_two.iloc[i,5],last_two.iloc[i,4],1)
    rat_dic[last_two.iloc[i,3]]["tr2"],rat_dic[last_two.iloc[i,2]]["tr-2"]=spreadelo(rat_dic[last_two.iloc[i,3]]["tr2"],rat_dic[last_two.iloc[i,2]]["tr-2"],60,last_two.iloc[i,5],last_two.iloc[i,4],2)
    rat_dic[last_two.iloc[i,2]]["ar"],rat_dic[last_two.iloc[i,3]]["dr"]=adelo(rat_dic[last_two.iloc[i,2]]["ar"],rat_dic[last_two.iloc[i,3]]["dr"],60,last_two.iloc[i,4],0)
    rat_dic[last_two.iloc[i,3]]["ar"],rat_dic[last_two.iloc[i,2]]["dr"]=adelo(rat_dic[last_two.iloc[i,3]]["ar"],rat_dic[last_two.iloc[i,2]]["dr"],60,last_two.iloc[i,5],0)
    rat_dic[last_two.iloc[i,2]]["ar1"],rat_dic[last_two.iloc[i,3]]["dr1"]=adelo(rat_dic[last_two.iloc[i,2]]["ar1"],rat_dic[last_two.iloc[i,3]]["dr1"],60,last_two.iloc[i,4],1)
    rat_dic[last_two.iloc[i,3]]["ar1"],rat_dic[last_two.iloc[i,2]]["dr1"]=adelo(rat_dic[last_two.iloc[i,3]]["ar1"],rat_dic[last_two.iloc[i,2]]["dr1"],60,last_two.iloc[i,5],1)    
    rat_dic[last_two.iloc[i,2]]["ar2"],rat_dic[last_two.iloc[i,3]]["dr2"]=adelo(rat_dic[last_two.iloc[i,2]]["ar2"],rat_dic[last_two.iloc[i,3]]["dr2"],60,last_two.iloc[i,4],2)
    rat_dic[last_two.iloc[i,3]]["ar2"],rat_dic[last_two.iloc[i,2]]["dr2"]=adelo(rat_dic[last_two.iloc[i,3]]["ar2"],rat_dic[last_two.iloc[i,2]]["dr2"],60,last_two.iloc[i,5],2)     
    rat_dic[last_two.iloc[i,2]]["kgv"],rat_dic[last_two.iloc[i,3]]["kgv"]=kgvelo(rat_dic[last_two.iloc[i,2]]["kgv"],rat_dic[last_two.iloc[i,3]]["kgv"],60,last_two.iloc[i,4],last_two.iloc[i,5])
    rat_dic[last_two.iloc[i,2]]["nm"]+=1
    rat_dic[last_two.iloc[i,3]]["nm"]+=1
    

#tar_set ga,gb,tg,s,kgv,oo,to,tho
for t in range(len(test)):
    tmp=[]
    tmp1=[]
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["tr"],rat_dic[test.iloc[t,3]]["tr"]))   
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["tr1"],rat_dic[test.iloc[t,3]]["tr-1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["tr2"],rat_dic[test.iloc[t,3]]["tr-2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["tr1"],rat_dic[test.iloc[t,2]]["tr-1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["tr2"],rat_dic[test.iloc[t,2]]["tr-2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["ar"],rat_dic[test.iloc[t,3]]["dr"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["ar"],rat_dic[test.iloc[t,2]]["dr"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["ar1"],rat_dic[test.iloc[t,3]]["dr1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["ar1"],rat_dic[test.iloc[t,2]]["dr1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["ar2"],rat_dic[test.iloc[t,3]]["dr2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["ar2"],rat_dic[test.iloc[t,2]]["dr2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["kgv"],rat_dic[test.iloc[t,3]]["kgv"]))
    tmp1.append(test.iloc[t,4])
    tmp1.append(test.iloc[t,5])
    tmp1.append(test.iloc[t,4]+test.iloc[t,5])
    tmp1.append(test.iloc[t,4]-test.iloc[t,5])
    tmp1.append((test.iloc[t,4]!=0) and (test.iloc[t,5]!=0))
    tmp1.append(1*((test.iloc[t,4]+test.iloc[t,5])>1.5))
    tmp1.append(1*((test.iloc[t,4]+test.iloc[t,5])>2.5))
    tmp1.append(1*((test.iloc[t,4]+test.iloc[t,5])>3.5))
    fet_set.append(np.array(tmp))
    tar_set.append(np.array(tmp1))

In [ ]:
df1['Date']=pd.to_datetime(df1['Date']
                              ,infer_datetime_format=True)
last_two1=df1[(df1["Date"].dt.year>2012) & (df1["Date"].dt.year<2018)]
for i in df1.Home.unique():
    rat_dic[i]={"tr":1500,"tr1":1500,"tr-1":1500,"tr2":1500,"tr-2":1500,"ar":1500,"ar1":1500,"ar2":1500,"dr":1500,"dr1":1500,"dr2":1500,"kgv":1500,"nm":0}

for i in df1.Away.unique():
    rat_dic[i]={"tr":1500,"tr1":1500,"tr-1":1500,"tr2":1500,"tr-2":1500,"ar":1500,"ar1":1500,"ar2":1500,"dr":1500,"dr1":1500,"dr2":1500,"kgv":1500,"nm":0}

test1=df1[(df1["Date"].dt.year>=2014) & (df1["Date"].dt.year<2019)]

for i in range(len(last_two1)):
    rat_dic[last_two1.iloc[i,5]]["tr"],rat_dic[last_two1.iloc[i,6]]["tr"]=spreadelo(rat_dic[last_two1.iloc[i,5]]["tr"],rat_dic[last_two1.iloc[i,6]]["tr"],60,last_two1.iloc[i,7],last_two1.iloc[i,8],0)
    rat_dic[last_two1.iloc[i,5]]["tr1"],rat_dic[last_two1.iloc[i,6]]["tr-1"]=spreadelo(rat_dic[last_two1.iloc[i,5]]["tr1"],rat_dic[last_two1.iloc[i,6]]["tr-1"],60,last_two1.iloc[i,7],last_two1.iloc[i,8],1)
    rat_dic[last_two1.iloc[i,5]]["tr2"],rat_dic[last_two1.iloc[i,6]]["tr-2"]=spreadelo(rat_dic[last_two1.iloc[i,5]]["tr2"],rat_dic[last_two1.iloc[i,6]]["tr-2"],60,last_two1.iloc[i,7],last_two1.iloc[i,8],2)
    rat_dic[last_two1.iloc[i,6]]["tr1"],rat_dic[last_two1.iloc[i,5]]["tr-1"]=spreadelo(rat_dic[last_two1.iloc[i,6]]["tr1"],rat_dic[last_two1.iloc[i,5]]["tr-1"],60,last_two1.iloc[i,8],last_two1.iloc[i,7],1)
    rat_dic[last_two1.iloc[i,6]]["tr2"],rat_dic[last_two1.iloc[i,5]]["tr-2"]=spreadelo(rat_dic[last_two1.iloc[i,6]]["tr2"],rat_dic[last_two1.iloc[i,5]]["tr-2"],60,last_two1.iloc[i,8],last_two1.iloc[i,7],2)
    rat_dic[last_two1.iloc[i,5]]["ar"],rat_dic[last_two1.iloc[i,6]]["dr"]=adelo(rat_dic[last_two1.iloc[i,5]]["ar"],rat_dic[last_two1.iloc[i,6]]["dr"],60,last_two1.iloc[i,7],0)
    rat_dic[last_two1.iloc[i,6]]["ar"],rat_dic[last_two1.iloc[i,5]]["dr"]=adelo(rat_dic[last_two1.iloc[i,6]]["ar"],rat_dic[last_two1.iloc[i,5]]["dr"],60,last_two1.iloc[i,8],0)
    rat_dic[last_two1.iloc[i,5]]["ar1"],rat_dic[last_two1.iloc[i,6]]["dr1"]=adelo(rat_dic[last_two1.iloc[i,5]]["ar1"],rat_dic[last_two1.iloc[i,6]]["dr1"],60,last_two1.iloc[i,7],1)
    rat_dic[last_two1.iloc[i,6]]["ar1"],rat_dic[last_two1.iloc[i,5]]["dr1"]=adelo(rat_dic[last_two1.iloc[i,6]]["ar1"],rat_dic[last_two1.iloc[i,5]]["dr1"],60,last_two1.iloc[i,8],1)    
    rat_dic[last_two1.iloc[i,5]]["ar2"],rat_dic[last_two1.iloc[i,6]]["dr2"]=adelo(rat_dic[last_two1.iloc[i,5]]["ar2"],rat_dic[last_two1.iloc[i,6]]["dr2"],60,last_two1.iloc[i,7],2)
    rat_dic[last_two1.iloc[i,6]]["ar2"],rat_dic[last_two1.iloc[i,5]]["dr2"]=adelo(rat_dic[last_two1.iloc[i,6]]["ar2"],rat_dic[last_two1.iloc[i,5]]["dr2"],60,last_two1.iloc[i,8],2)     
    rat_dic[last_two1.iloc[i,5]]["kgv"],rat_dic[last_two1.iloc[i,6]]["kgv"]=kgvelo(rat_dic[last_two1.iloc[i,5]]["kgv"],rat_dic[last_two1.iloc[i,6]]["kgv"],60,last_two1.iloc[i,7],last_two1.iloc[i,8])
    rat_dic[last_two1.iloc[i,5]]["nm"]+=1
    rat_dic[last_two1.iloc[i,6]]["nm"]+=1
    
#tar_set ga,gb,tg,s,kgv,oo,to,tho
for t in range(len(test1)):
    tmp=[]
    tmp1=[]
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["tr"],rat_dic[test1.iloc[t,6]]["tr"]))   
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["tr1"],rat_dic[test1.iloc[t,6]]["tr-1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["tr2"],rat_dic[test1.iloc[t,6]]["tr-2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["tr1"],rat_dic[test1.iloc[t,5]]["tr-1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["tr2"],rat_dic[test1.iloc[t,5]]["tr-2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["ar"],rat_dic[test1.iloc[t,6]]["dr"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["ar"],rat_dic[test1.iloc[t,5]]["dr"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["ar1"],rat_dic[test1.iloc[t,6]]["dr1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["ar1"],rat_dic[test1.iloc[t,5]]["dr1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["ar2"],rat_dic[test1.iloc[t,6]]["dr2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["ar2"],rat_dic[test1.iloc[t,5]]["dr2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["kgv"],rat_dic[test1.iloc[t,6]]["kgv"]))
    tmp1.append(test1.iloc[t,7])
    tmp1.append(test1.iloc[t,8])
    tmp1.append(test1.iloc[t,7]+test1.iloc[t,8])
    tmp1.append(test1.iloc[t,7]-test1.iloc[t,8])
    tmp1.append((test1.iloc[t,7]!=0) and (test1.iloc[t,8]!=0))
    tmp1.append(1*((test1.iloc[t,7]+test1.iloc[t,8])>1.5))
    tmp1.append(1*((test1.iloc[t,7]+test1.iloc[t,8])>2.5))
    tmp1.append(1*((test1.iloc[t,7]+test1.iloc[t,8])>3.5))
    fet_set.append(np.array(tmp))
    tar_set.append(np.array(tmp1))



In [37]:
fet_set=list(fet_set)[:-n]
tar_set=list(tar_set)[:-n]

In [66]:
n=40
for i in range(n):
    t1=df[(df["Date"].dt.year==2019) ].iloc[i,2]
    t2=df[(df["Date"].dt.year==2019) ].iloc[i,3]
    tmp=[]
    
    tmp1=[]
#t1="Anderlecht"
#t2="Oostende"
    tmp.append(nprob(rat_dic[t1]["tr"],rat_dic[t2]["tr"]))   
    tmp.append(nprob(rat_dic[t1]["tr1"],rat_dic[t2]["tr-1"]))
    tmp.append(nprob(rat_dic[t1]["tr2"],rat_dic[t2]["tr-2"]))
    tmp.append(nprob(rat_dic[t2]["tr1"],rat_dic[t1]["tr-1"]))
    tmp.append(nprob(rat_dic[t2]["tr2"],rat_dic[t1]["tr-2"]))
    tmp.append(nprob(rat_dic[t1]["ar"],rat_dic[t2]["dr"]))
    tmp.append(nprob(rat_dic[t2]["ar"],rat_dic[t1]["dr"]))
    tmp.append(nprob(rat_dic[t1]["ar1"],rat_dic[t2]["dr1"]))
    tmp.append(nprob(rat_dic[t2]["ar1"],rat_dic[t1]["dr1"]))
    tmp.append(nprob(rat_dic[t1]["ar2"],rat_dic[t2]["dr2"]))
    tmp.append(nprob(rat_dic[t2]["ar2"],rat_dic[t1]["dr2"]))
    tmp.append(nprob(rat_dic[t1]["kgv"],rat_dic[t2]["kgv"]))
    fet_set.append(tmp)
    tmp1.append(test.iloc[i,4])
    tmp1.append(test.iloc[i,5])
    tmp1.append(test.iloc[i,4]+test.iloc[i,5])
    tmp1.append(test.iloc[i,4]-test.iloc[i,5])
    tmp1.append((test.iloc[i,4]!=0) and (test.iloc[i,5]!=0))
    tmp1.append(1*((test.iloc[i,4]+test.iloc[i,5])>1.5))
    tmp1.append(1*((test.iloc[i,4]+test.iloc[i,5])>2.5))
    tmp1.append(1*((test.iloc[i,4]+test.iloc[i,5])>3.5))
    tar_set.append(np.array(tmp1))
    print(t1,t2,tmp1,i)

Aston Villa QPR [1.0, 1.0, 2.0, 0.0, True, 1, 0, 0] 0
Blackburn West Brom [2.0, 2.0, 4.0, 0.0, True, 1, 1, 1] 1
Brentford Norwich [1.0, 1.0, 2.0, 0.0, True, 1, 0, 0] 2
Derby Middlesbrough [3.0, 2.0, 5.0, 1.0, True, 1, 1, 1] 3
Hull Bolton [0.0, 1.0, 1.0, -1.0, False, 0, 0, 0] 4
Ipswich Millwall [1.0, 1.0, 2.0, 0.0, True, 1, 0, 0] 5
Nott'm Forest Leeds [1.0, 2.0, 3.0, -1.0, True, 1, 1, 0] 6
Reading Swansea [1.0, 3.0, 4.0, -2.0, True, 1, 1, 1] 7
Rotherham Preston [2.0, 1.0, 3.0, 1.0, True, 1, 1, 0] 8
Sheffield Weds Birmingham [1.0, 1.0, 2.0, 0.0, True, 1, 0, 0] 9
Stoke Bristol City [2.0, 0.0, 2.0, 2.0, False, 1, 0, 0] 10
Wigan Sheffield United [4.0, 1.0, 5.0, 3.0, True, 1, 1, 1] 11
Leeds Derby [1.0, 0.0, 1.0, 1.0, False, 0, 0, 0] 12
Birmingham Middlesbrough [0.0, 2.0, 2.0, -2.0, False, 1, 0, 0] 13
Brentford Stoke [1.0, 1.0, 2.0, 0.0, True, 1, 0, 0] 14
Bristol City Bolton [1.0, 0.0, 1.0, 1.0, False, 0, 0, 0] 15
Hull Sheffield Weds [1.0, 0.0, 1.0, 1.0, False, 0, 0, 0] 16
Ipswich Rotherham [

In [187]:
clustering = SpectralClustering(n_clusters=200,n_neighbors=2,assign_labels="kmeans",random_state=0).fit(fet_set)

In [69]:
brc.labels_

array([ 50, 121,  85, ...,  75, 136,  73])

In [18]:
fet_set[0]

array([0.59671478, 0.07743595, 0.03748268, 0.25311374, 0.02995152,
       0.61956499, 0.57384109, 0.10989534, 0.21635694, 0.0475472 ,
       0.06128603, 0.40982078])

In [43]:
f=[[] for i in range(200)]
for i in range(len(clustering.labels_)):
    f[clustering.labels_[i]].append(tar_set[i])

In [36]:
f[5]

[array([1., 1., 2., 0., 1., 1., 0., 0.])]

In [46]:
for i in range(1,n+1):
    print(f[clustering.labels_[-i]],"\n")

[array([1., 0., 1., 1., 0., 0., 0., 0.]), array([1., 0., 1., 1., 0., 0., 0., 0.]), array([ 1.,  4.,  5., -3.,  1.,  1.,  1.,  1.]), array([0., 0., 0., 0., 0., 0., 0., 0.]), array([2., 1., 3., 1., 1., 1., 1., 0.]), array([3., 0., 3., 3., 0., 1., 1., 0.]), array([ 0.,  1.,  1., -1.,  0.,  0.,  0.,  0.]), array([1., 1., 2., 0., 1., 1., 0., 0.]), array([2., 1., 3., 1., 1., 1., 1., 0.]), array([1., 0., 1., 1., 0., 0., 0., 0.]), array([4., 0., 4., 4., 0., 1., 1., 1.]), array([4., 3., 7., 1., 1., 1., 1., 1.]), array([ 1.,  2.,  3., -1.,  1.,  1.,  1.,  0.]), array([1., 0., 1., 1., 0., 0., 0., 0.]), array([5., 0., 5., 5., 0., 1., 1., 1.]), array([1., 0., 1., 1., 0., 0., 0., 0.]), array([2., 1., 3., 1., 1., 1., 1., 0.]), array([2., 2., 4., 0., 1., 1., 1., 1.]), array([ 1.,  2.,  3., -1.,  1.,  1.,  1.,  0.]), array([ 0.,  2.,  2., -2.,  0.,  1.,  0.,  0.]), array([ 1.,  2.,  3., -1.,  1.,  1.,  1.,  0.])] 

[array([ 0.,  1.,  1., -1.,  0.,  0.,  0.,  0.]), array([1., 1., 2., 0., 1., 1., 0., 0.]

In [50]:
np.unique(np.array(f[0]))

array([-2., -1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.])

In [172]:
brc = Birch(branching_factor=50, n_clusters=None, threshold=0.2,compute_labels=True).fit(np.array(fet_set))


In [188]:
for r in range(1,41):
    dic={}
    dic1={}
    i=clustering.labels_[-r]
    for j in range(len(f[i])):
        if(str(f[i][j]) not in dic):
            dic[str(f[i][j])]=f[i][j]
        if(str(f[i][j]) not in dic1):
            dic1[str(f[i][j])]=1
        else:
            dic1[str(f[i][j])]+=1

    print([[dic[i],dic1[i]] for i in [k for (k,v) in dic1.items() ]],"\n----------------------\n",tar_set[-r],"\n======================\n")

[[array([3., 3., 6., 0., 1., 1., 1., 1.]), 1], [array([ 0.,  1.,  1., -1.,  0.,  0.,  0.,  0.]), 3], [array([4., 0., 4., 4., 0., 1., 1., 1.]), 1], [array([1., 1., 2., 0., 1., 1., 0., 0.]), 2], [array([1., 0., 1., 1., 0., 0., 0., 0.]), 4], [array([0., 0., 0., 0., 0., 0., 0., 0.]), 1], [array([2., 1., 3., 1., 1., 1., 1., 0.]), 1]] 
----------------------
 [ 1.  2.  3. -1.  1.  1.  1.  0.] 

[[array([ 0.,  1.,  1., -1.,  0.,  0.,  0.,  0.]), 1]] 
----------------------
 [2. 2. 4. 0. 1. 1. 1. 1.] 

[[array([2., 2., 4., 0., 1., 1., 1., 1.]), 3], [array([ 0.,  3.,  3., -3.,  0.,  1.,  1.,  0.]), 1], [array([ 0.,  2.,  2., -2.,  0.,  1.,  0.,  0.]), 3], [array([5., 0., 5., 5., 0., 1., 1., 1.]), 1], [array([3., 0., 3., 3., 0., 1., 1., 0.]), 3], [array([0., 0., 0., 0., 0., 0., 0., 0.]), 1], [array([2., 1., 3., 1., 1., 1., 1., 0.]), 4], [array([ 0.,  1.,  1., -1.,  0.,  0.,  0.,  0.]), 4], [array([1., 1., 2., 0., 1., 1., 0., 0.]), 2], [array([3., 1., 4., 2., 1., 1., 1., 1.]), 1], [array([2., 0.,

[[array([3., 1., 4., 2., 1., 1., 1., 1.]), 1]] 
----------------------
 [1. 0. 1. 1. 0. 0. 0. 0.] 

[] 
----------------------
 [4. 1. 5. 3. 1. 1. 1. 1.] 

[[array([ 1.,  3.,  4., -2.,  1.,  1.,  1.,  1.]), 1], [array([1., 1., 2., 0., 1., 1., 0., 0.]), 2], [array([2., 1., 3., 1., 1., 1., 1., 0.]), 1], [array([2., 0., 2., 2., 0., 1., 0., 0.]), 1], [array([1., 0., 1., 1., 0., 0., 0., 0.]), 1], [array([0., 0., 0., 0., 0., 0., 0., 0.]), 1]] 
----------------------
 [2. 0. 2. 2. 0. 1. 0. 0.] 

[[array([1., 0., 1., 1., 0., 0., 0., 0.]), 8], [array([4., 1., 5., 3., 1., 1., 1., 1.]), 1], [array([3., 1., 4., 2., 1., 1., 1., 1.]), 5], [array([2., 2., 4., 0., 1., 1., 1., 1.]), 4], [array([2., 0., 2., 2., 0., 1., 0., 0.]), 10], [array([1., 1., 2., 0., 1., 1., 0., 0.]), 12], [array([ 1.,  3.,  4., -2.,  1.,  1.,  1.,  1.]), 2], [array([0., 0., 0., 0., 0., 0., 0., 0.]), 5], [array([4., 2., 6., 2., 1., 1., 1., 1.]), 2], [array([ 0.,  1.,  1., -1.,  0.,  0.,  0.,  0.]), 5], [array([ 0.,  2.,  2., -2.,